In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

In [3]:
from utils import load_test_cases
from my_notion_companion.retriever import SelfQueryAgent, BasicRetriever

In [4]:
from langchain_community.llms import LlamaCpp

import tomllib

with open('../.config.toml', 'rb') as f:
    _CONFIGS = tomllib.load(f)

with open('../.tokens.toml', 'rb') as f:
    _TOKENS = tomllib.load(f)

llm = LlamaCpp(
    model_path=_CONFIGS['model_path']+'/'+_CONFIGS['model_mapping'][_CONFIGS['model_name']],
    name=_CONFIGS['model_name'], 
    **_CONFIGS['llm']
)

/Users/fred/micromamba/envs/my-notion-companion/lib/python3.12/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! conversation is not default parameter.
                conversation was transferred to model_kwargs.
                Please confirm that conversation is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 21 key-value pairs and 387 tensors from /Users/fred/Documents/models/qwen1_5-7b-chat-q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.name str              = Qwen1.5-7B-Chat-AWQ-fp16
llama_model_loader: - kv   2:                          qwen2.block_count u32              = 32
llama_model_loader: - kv   3:                       qwen2.context_length u32              = 32768
llama_m

In [5]:
test_cases = load_test_cases('../data/test_cases.txt')
test_cases[:3]

[{'q': '什么是我国第一部编年史著作？', 'a': '《左传》。', 'docs': ['附：《左传》是我国第一部编年史著作。\n']},
 {'q': '什么是我国第一部编年国别史？', 'a': '《国语》。', 'docs': ['附：《国语》是我国第一部编年国别史。\n']},
 {'q': '“寡人之于国也”下一句是什么？来自哪里？',
  'a': '“寡人之于国也”下一句是“尽心焉耳矣”。这个句子来自《孟子》。',
  'docs': ['梁惠王曰：“寡人之于国也，尽心焉耳矣。河内凶，则移其民于河东，移其粟于河内；河东凶亦然。察邻国之政，无如寡人之用心者。邻国之民不加少，寡人之民不加多，何也？”',
   '《寡人之于国也》（孟子）\n']}]

In [6]:
len(test_cases)

37

In [7]:
from tqdm import tqdm
import re

def match_chinese(string_to_match, docs):
    # find all chinese characters
    # ref: https://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex
    for tokens in re.findall(r'[\u4e00-\u9fff]+', string_to_match):
        return any([tokens in x.page_content for x in docs])

def evaluate_retriever(test_cases, retriever):
    score_list = list()

    for case in tqdm(test_cases):
        docs_retrieved = retriever.invoke(case['q'])
        contains_all_ref = list()
        for ref in case['docs']:
            contains_all_ref.append(match_chinese(ref, docs_retrieved))
            
        
        score_list.extend(contains_all_ref)

    return sum(score_list)/len(score_list), score_list

In [12]:
%%time

# only retriever
stats, score_list = evaluate_retriever(test_cases,  BasicRetriever(_CONFIGS, _TOKENS))

  0%|                                                                                                           | 0/37 [00:00<?, ?it/s]score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags

CPU times: user 1.02 s, sys: 109 ms, total: 1.13 s
Wall time: 3.25 s


In [13]:
stats

0.1836734693877551

In [10]:
from my_notion_companion.query_constructor import QueryConstructor
from langchain_core.runnables import RunnableLambda

qc = QueryConstructor(llm, _CONFIGS)
b = BasicRetriever(_CONFIGS, _TOKENS)

retriever_plus = RunnableLambda(qc.invoke) | RunnableLambda(b.invoke)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [16]:
%%time
# with query constructor + basic retriever
stats, score_list = evaluate_retriever(test_cases, retriever_plus)

  0%|                                                                                                           | 0/37 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    6547.34 ms
llama_print_timings:      sample time =      11.42 ms /    32 runs   (    0.36 ms per token,  2803.08 tokens per second)
llama_print_timings: prompt eval time =     409.05 ms /    24 tokens (   17.04 ms per token,    58.67 tokens per second)
llama_print_timings:        eval time =    1039.13 ms /    31 runs   (   33.52 ms per token,    29.83 tokens per second)
llama_print_timings:       total time =    1647.75 ms /    55 tokens
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'd

CPU times: user 1min 28s, sys: 2.66 s, total: 1min 31s
Wall time: 1min 57s


In [17]:
stats

0.20408163265306123

In [12]:
%%time

stats, score_list = evaluate_retriever(test_cases,  SelfQueryAgent(llm, _CONFIGS, _TOKENS))


llama_print_timings:        load time =    8357.14 ms
llama_print_timings:      sample time =      91.75 ms /   256 runs   (    0.36 ms per token,  2790.25 tokens per second)
llama_print_timings: prompt eval time =    8354.64 ms /  1134 tokens (    7.37 ms per token,   135.73 tokens per second)
llama_print_timings:        eval time =   12966.16 ms /   255 runs   (   50.85 ms per token,    19.67 tokens per second)
llama_print_timings:       total time =   23012.52 ms /  1389 tokens
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 'date_end']
Metadata key date_end not found in metadata. Setting to None. 
Metadata fields defined for this instance: ['author', 'id', 'name', 'source', 'tags', 'date_start', 

CPU times: user 26min 16s, sys: 3min 26s, total: 29min 43s
Wall time: 39min 10s


In [13]:
stats

0.0625

In [1]:
import pickle
with open('../data/notion_offline.pkl', 'rb') as f:
    docs_from_notion = pickle.load(f)

In [2]:
docs = list()
for k, v in docs_from_notion.items():
    docs.extend(v)

len(docs)

225

In [13]:
l = [1,2,3, 4, 5]
l[1,2,3]

TypeError: list indices must be integers or slices, not tuple

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

chunk_params = {
    # 1 Chinese characters = 2 english character
    # most paragraphs/sections are within 500 Chinese words/chars
    'chunk_size': 500, 
    'chunk_overlap': 100,
}

rc_splitter = RecursiveCharacterTextSplitter(**chunk_params)

splits = rc_splitter.split_documents(docs)
len(docs_from_notion), len(splits)

(4, 2282)

In [27]:
from langchain.retrievers import BM25Retriever
import jieba

# ref: https://www.kaggle.com/code/terrychanorg/rank-bm25-notebook34e931b070
# ref: https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/retrievers/bm25.py#L15
# ref: https://github.com/fxsjy/jieba?tab=readme-ov-file
retriever = BM25Retriever.from_documents(
    splits,
    preprocess_func=lambda x: jieba.lcut_for_search(x, HMM=False)
)

In [28]:
%%time

stats, score_list = evaluate_retriever(test_cases,  retriever)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 188.78it/s]

CPU times: user 193 ms, sys: 1.5 ms, total: 195 ms
Wall time: 197 ms


In [29]:
stats

0.3877551020408163

In [30]:
from my_notion_companion.query_constructor import QueryConstructor
from langchain_core.runnables import RunnableLambda

qc = QueryConstructor(llm, _CONFIGS)

retriever_plus = RunnableLambda(qc.invoke) | retriever

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [31]:
%%time

stats, score_list = evaluate_retriever(test_cases,  retriever_plus)

  0%|                                                                                                           | 0/37 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    6547.34 ms
llama_print_timings:      sample time =       2.65 ms /     7 runs   (    0.38 ms per token,  2640.51 tokens per second)
llama_print_timings: prompt eval time =    1081.35 ms /     7 tokens (  154.48 ms per token,     6.47 tokens per second)
llama_print_timings:        eval time =     200.03 ms /     6 runs   (   33.34 ms per token,    30.00 tokens per second)
llama_print_timings:       total time =    1330.87 ms /    13 tokens
  3%|██▋                                                                                                | 1/37 [00:01<00:48,  1.35s/it]Llama.generate: prefix-match hit

llama_print_timings:        load time =    6547.34 ms
llama_print_timings:      sample time =       3.16 ms /     9 runs   (    0.35 ms per token,  2850.81 tokens per second)
l

CPU times: user 1min 27s, sys: 5.86 s, total: 1min 33s
Wall time: 1min 26s


In [32]:
stats

0.46938775510204084

In [33]:
from langchain_community.retrievers import TFIDFRetriever

retriever = TFIDFRetriever.from_documents(splits)

In [34]:
%%time

stats, score_list = evaluate_retriever(test_cases,  retriever)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 39.30it/s]

CPU times: user 852 ms, sys: 61.3 ms, total: 913 ms
Wall time: 942 ms


In [35]:
stats

0.30612244897959184

In [36]:
from my_notion_companion.query_constructor import QueryConstructor
from langchain_core.runnables import RunnableLambda

qc = QueryConstructor(llm, _CONFIGS)

retriever_plus = RunnableLambda(qc.invoke) | retriever

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [37]:
%%time

stats, score_list = evaluate_retriever(test_cases,  retriever_plus)

  0%|                                                                                                           | 0/37 [00:00<?, ?it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    6547.34 ms
llama_print_timings:      sample time =       2.55 ms /     7 runs   (    0.36 ms per token,  2741.87 tokens per second)
llama_print_timings: prompt eval time =     354.81 ms /     7 tokens (   50.69 ms per token,    19.73 tokens per second)
llama_print_timings:        eval time =     198.40 ms /     6 runs   (   33.07 ms per token,    30.24 tokens per second)
llama_print_timings:       total time =     599.99 ms /    13 tokens
  3%|██▋                                                                                                | 1/37 [00:00<00:23,  1.52it/s]Llama.generate: prefix-match hit

llama_print_timings:        load time =    6547.34 ms
llama_print_timings:      sample time =       3.49 ms /     9 runs   (    0.39 ms per token,  2576.58 tokens per second)
l

CPU times: user 1min 28s, sys: 6.39 s, total: 1min 35s
Wall time: 1min 28s


In [38]:
stats

0.2857142857142857

In [42]:
def _pretty_print(docs):
    for doc in docs:
        print(doc.metadata)
        print(doc.page_content)
        print("\n" + "-" * 30)

In [110]:
i = -5
case = test_cases[i]

docs_retrieved = retriever.invoke(case['q'])
print(_pretty_print(docs_retrieved))
# print(docs_retrieved)
print(case)
print("-"*20)
for ref in case['docs']:
    print(match_chinese(ref, docs_retrieved))

{'author': '孔子', 'date': {'start': '2019-05-16', 'end': None, 'time_zone': None}, 'tags': ['古文'], 'name': '论语', 'id': 'f27c7f42-c7c1-4e24-bee5-0fcbdccd1cd1'}
1.6 子曰：“弟子入则孝，出则弟，谨而信，泛爱众，而亲仁。行有余力，则以学文。”

1.7 子夏曰：“贤贤易色；事父母，能竭其力；事君，能致其身；与朋友交，言而有信。虽曰未学，吾必谓之学矣。”

1.8 子曰：“君子不重则不威，学则不固。主忠信，无友不如己者，过则勿惮改。”
【不如己者：与自己志趣不同的人，而不应理解为不如自己的人。这五句话没有什么联系，可能是孔子在不同场合说的。】

1.10 子禽问于子贡曰：“夫子至于是邦也，必闻其政，求之与，抑与之与？”子贡曰：“夫子温、良、恭、俭、让以得之。夫子之求之也，其诸异乎人之求之与？”

1.12 有子曰：“礼之用，和为贵。先王之道，斯为美，小大由之。有所不行，知和而和，不以礼节之，亦不可行也。”

1.14 子曰：“君子食无求饱，居无求安，敏于事而慎于言，就有道而正焉。可谓好学也已。”

------------------------------
{'author': None, 'date': {'start': '2013-08-12', 'end': None, 'time_zone': None}, 'tags': ['古文'], 'name': '高中古诗文', 'id': '0c77a756-a81d-486d-83de-2de6fb7e9e5d'}
《子路、曾皙、冉有、公西华侍坐》（论语）
子路、曾皙、冉有、公西华侍坐。子曰：“以吾一日长乎尔，毋吾以也。居则曰：‘不吾知也。’如或知尔，则何以哉？”
子路率尔而对曰：“千乘之国，摄乎大国之间，加之以师旅，因之以饥馑；由也为之，比及三年，可使有勇，且知方也。”
夫子哂（shěn）之。
“求，尔何如？”
对曰：“方六七十，如五六十，求也为之，比及三年，可使足民。如其礼乐，以俟君子。”
“赤，尔何如？”
对曰：“非曰能之，愿学焉。宗庙之事，如会同，端章甫，愿为小相焉。”
“点，尔何如？”
鼓瑟希，铿尔，舍瑟而作，对曰：“异乎三子者之撰。”
子曰：“